# Is there a significant difference between the proportion of rent burdened households vs mortgage burdened households?

In [1]:
import statsmodels.stats.proportion as stats
import pandas as pd
import geopandas as gpd

rent = gpd.read_file('../data/processed/q1Map_in.geojson')
mortgage = pd.read_csv('../data/processed/mortgage_ratio.csv')

ERROR 1: PROJ: proj_create_from_database: Open of /home/colin/anaconda3/envs/capstone/share/proj failed


In [2]:
mortgage = mortgage[['L_HOOD','Owner_Households','Total_Under_25']]
print(mortgage.columns)
print(mortgage.shape)

Index(['L_HOOD', 'Owner_Households', 'Total_Under_25'], dtype='object')
(18, 3)


In [3]:
rent = rent[['L_HOOD','Renter_Households','Total_under_35']]
print(rent.columns)
print(rent.shape)

Index(['L_HOOD', 'Renter_Households', 'Total_under_35'], dtype='object')
(20, 3)


In [4]:
merged = pd.merge(rent, mortgage, on='L_HOOD')
merged.head()

,L_HOOD,Renter_Households,Total_under_35,Owner_Households,Total_Under_25
0,Ballard,16025.0,9496.0,9270,4835
1,Beacon Hill,1647.0,666.0,1651,591
2,Capitol Hill,7812.0,4764.0,6184,2795
3,Cascade,10459.0,7052.0,1323,616
4,Central Area,1829.0,851.0,508,157


In [14]:
from statsmodels.stats.proportion import proportions_ztest
alpha = 0.05
sig = pd.DataFrame(columns=["Neighborhood","Difference","Significant"])

for index,row in merged.iterrows():
    # renter proportion - homeowner proportion
    difference = (row['Total_under_35']/row['Renter_Households']) - (row['Total_Under_25']/row['Owner_Households'])

    success = [row['Total_under_35'],row['Total_Under_25']]
    attampts = [row['Renter_Households'],row['Owner_Households']]
    # proportion_ztest([Array of successes], [Array of attemps])
    z_score, p_value = proportions_ztest(success,attampts)
    sig.loc[len(sig)] = [row['L_HOOD'],difference,(p_value < 0.05)]
    print(f"{row['L_HOOD']}: {p_value}")
sig

Ballard: 4.790435929388905e-28
Beacon Hill: 0.006075018280392457
Capitol Hill: 2.7425984369737735e-77
Cascade: 7.439390355117222e-51
Central Area: 3.180812681480587e-10
Delridge: 7.056896644982367e-36
Downtown: 9.238952952574035e-16
Interbay: 3.992250084526412e-143
Greater Duwamish: 2.1138925130348556e-10
Northeast: 0.00410870260052756
Northgate: 9.87536747868575e-38
Queen Anne: 1.2608049835865371e-58
Rainier Valley: 0.44909513243355303
Seward Park: 3.0082895862301163e-12
West Seattle: 5.371756725440816e-16
Northwest: 3.004628592194978e-08
North Central: 2.079277813067461e-10
University District: 5.379768475971336e-06


,Neighborhood,Difference,Significant
0,Ballard,0.070999,True
1,Beacon Hill,0.046407,True
2,Capitol Hill,0.157858,True
3,Cascade,0.208643,True
4,Central Area,0.156226,True
5,Delridge,-0.103224,True
6,Downtown,0.073182,True
7,Interbay,0.327918,True
8,Greater Duwamish,-0.104483,True
9,Northeast,-0.072393,True
